In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
from iminuit import Minuit
from iminuit.cost import LeastSquares
import pickle
import uproot as ur
import re

In [28]:
input_file = f"../../genIII_data/Sim/e+_2.5_0deg.edm4hep.root"

events = ur.open("%s:events"%(input_file))

In [29]:
print(events["HcalFarForwardZDCHits"].keys())

['HcalFarForwardZDCHits.cellID', 'HcalFarForwardZDCHits.energy', 'HcalFarForwardZDCHits.position.x', 'HcalFarForwardZDCHits.position.y', 'HcalFarForwardZDCHits.position.z', 'HcalFarForwardZDCHits.contributions_begin', 'HcalFarForwardZDCHits.contributions_end']


In [30]:
def map_z_to_layer(z_array_list):
    # Flatten the array of arrays
    all_z = np.concatenate(z_array_list)
    
    # Find unique sorted z-values
    unique_z = np.unique(all_z)
    
    # Create a mapping from z to layer index
    z_to_layer = {z: i for i, z in enumerate(unique_z)}
    
    return z_to_layer

import numpy as np

def map_channels_by_layer(x_array_list, y_array_list, z_array_list, z_to_layer):
    # Flatten arrays
    x_all = np.concatenate(x_array_list)
    y_all = np.concatenate(y_array_list)
    z_all = np.concatenate(z_array_list)

    # Get corresponding layers
    layers = np.array([z_to_layer[z] for z in z_all])

    # Function to map positions to channel numbers
    def get_channel_map(x_vals, y_vals):
        # Find unique x and y positions
        unique_x = np.unique(x_vals)
        unique_y = np.unique(y_vals)

        # Sort from right to left (descending x) and bottom to top (ascending y)
        sorted_x = np.sort(unique_x)  # right to left
        sorted_y = np.sort(unique_y)        # bottom to top

        # Create mapping for grid positions
        channel_map = {}
        ch = 0
        for y in sorted_y:
            for x in sorted_x:
                channel_map[(x, y)] = ch
                ch += 1
        return channel_map

    # Separate into even and odd layer hits
    even_mask = (layers % 2 == 0)
    odd_mask  = (layers % 2 == 1)

    even_channel_map = get_channel_map(x_all[even_mask], y_all[even_mask])
    odd_channel_map  = get_channel_map(x_all[odd_mask], y_all[odd_mask])

    return even_channel_map, odd_channel_map


In [31]:
x_array_list = events["HcalFarForwardZDCHits"]["HcalFarForwardZDCHits.position.x"]
y_array_list = events["HcalFarForwardZDCHits"]["HcalFarForwardZDCHits.position.y"]
z_array_list = events["HcalFarForwardZDCHits"]["HcalFarForwardZDCHits.position.z"]

z_array_list = np.array(z_array_list)
z_to_layer_map = map_z_to_layer(z_array_list)

even_ch_map, odd_ch_map = map_channels_by_layer(np.array(x_array_list), np.array(y_array_list), z_array_list, z_to_layer_map)

In [32]:
print(even_ch_map)
print(odd_ch_map)
print(z_to_layer_map)

{(-85.4, -85.4): 0, (-36.6, -85.4): 1, (12.2, -85.4): 2, (61.0, -85.4): 3, (109.8, -85.4): 4, (-85.4, -36.6): 5, (-36.6, -36.6): 6, (12.2, -36.6): 7, (61.0, -36.6): 8, (109.8, -36.6): 9, (-85.4, 12.2): 10, (-36.6, 12.2): 11, (12.2, 12.2): 12, (61.0, 12.2): 13, (109.8, 12.2): 14, (-85.4, 61.0): 15, (-36.6, 61.0): 16, (12.2, 61.0): 17, (61.0, 61.0): 18, (109.8, 61.0): 19, (-85.4, 109.8): 20, (-36.6, 109.8): 21, (12.2, 109.8): 22, (61.0, 109.8): 23, (109.8, 109.8): 24}
{(-109.8, -109.8): 0, (-61.0, -109.8): 1, (-12.2, -109.8): 2, (36.6, -109.8): 3, (85.4, -109.8): 4, (-109.8, -61.0): 5, (-61.0, -61.0): 6, (-12.2, -61.0): 7, (36.6, -61.0): 8, (85.4, -61.0): 9, (-109.8, -12.2): 10, (-61.0, -12.2): 11, (-12.2, -12.2): 12, (36.6, -12.2): 13, (85.4, -12.2): 14, (-109.8, 36.6): 15, (-61.0, 36.6): 16, (-12.2, 36.6): 17, (36.6, 36.6): 18, (85.4, 36.6): 19, (-109.8, 85.4): 20, (-61.0, 85.4): 21, (-12.2, 85.4): 22, (36.6, 85.4): 23, (85.4, 85.4): 24}
{52.756: 0, 79.868: 1, 106.98: 2, 134.092: 3, 16

In [33]:
# Put all maps into a single dictionary for convenience
data_to_save = {
    "even_ch_map": even_ch_map,
    "odd_ch_map": odd_ch_map,
    "z_to_layer_map": z_to_layer_map
}

# Save to pickle file
with open("sim_geometry.pkl", "wb") as f:
    pickle.dump(data_to_save, f)